# Understanding Memory in LLMs

In the previous Notebook 03, we successfully explored how OpenAI models can enhance the results from Azure Cognitive Search. [Bing Chat](http://chat.bing.com/) is a search engine with a GPT-4 model that utilizes the content of search results to provide context and deliver accurate responses to queries.

However, we have yet to discover how to engage in a conversation with the LLM. With Bing Chat, this is possible, as the LLM can understand and reference the previous responses.

There is a common misconception that GPT models have memory. This is not true. While they possess knowledge, they do not retain information from previous questions asked to them.

The aim of this Notebook is to demonstrate how we can "provide memory" to the LLM by utilizing prompts and context.

In [1]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferMemory
from openai.error import OpenAIError
from langchain.docstore.document import Document
from langchain.memory import CosmosDBChatMessageHistory

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from common.utils import (
    get_search_results,
    order_search_results,
    model_tokens_limit,
    num_tokens_from_docs,
    embed_docs,
    search_docs,
    get_answer,
)

from common.prompts import COMBINE_QUESTION_PROMPT, COMBINE_PROMPT, COMBINE_CHAT_PROMPT

import logging

# Get the root logger
logger = logging.getLogger()
# Set the logging level to a higher level to ignore INFO messages
logger.setLevel(logging.WARNING)

# Don't mess with this unless you really know what you are doing
AZURE_SEARCH_API_VERSION = '2021-04-30-Preview'
AZURE_OPENAI_API_VERSION = "2023-03-15-preview"


# Change these below with your own services credentials
AZURE_SEARCH_ENDPOINT ="ENTER YOUR VALUE"
AZURE_SEARCH_KEY = "ENTER YOUR VALUE"
AZURE_OPENAI_ENDPOINT = "ENTER YOUR VALUE"
AZURE_OPENAI_API_KEY = "ENTER YOUR VALUE"
AZURE_COSMOSDB_ENDPOINT = "ENTER YOUR VALUE"
AZURE_COSMOSDB_NAME = "ENTER YOUR VALUE"
AZURE_COSMOSDB_CONTAINER_NAME = "ENTER YOUR VALUE"
AZURE_COMOSDB_CONNECTION_STRING = "ENTER YOUR VALUE" # Find this in the Keys section

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["AZURE_SEARCH_KEY"] = AZURE_SEARCH_KEY
os.environ["AZURE_SEARCH_ENDPOINT"] = AZURE_SEARCH_ENDPOINT
os.environ["AZURE_SEARCH_API_VERSION"] = AZURE_SEARCH_API_VERSION
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_COSMOSDB_ENDPOINT"]=  AZURE_COSMOSDB_ENDPOINT
os.environ["AZURE_COSMOSDB_NAME"]=  AZURE_COSMOSDB_NAME
os.environ["AZURE_COSMOSDB_CONTAINER_NAME"]=  AZURE_COSMOSDB_CONTAINER_NAME
os.environ["AZURE_COMOSDB_CONNECTION_STRING"]=  AZURE_COMOSDB_CONNECTION_STRING

### Let's start with the basics
Let's use a very simple example to see if the GPT model of Azure OpenAI have memory. We again will be using langchain to simplify our code 

In [3]:
QUESTION = "Tell me some use cases for reinforcement learning?"
FOLLOW_UP_QUESTION = "Can you rephrase what you just said?"

In [4]:
# Define model
MODEL = "gpt-35-turbo"
# Create an OpenAI instance
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0.5)

In [5]:
# We create a very simple prompt template, just the question as is:
prompt = PromptTemplate(
    input_variables=["question"],
    template="{question}",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [7]:
# Let's see what the GPT model responds
response = chain.run(QUESTION)
printmd(response)

1. Game playing: Reinforcement learning algorithms have been used to develop game playing agents that can learn to play games such as chess, Go, and poker.

2. Robotics: Reinforcement learning has been used to train robots to perform tasks such as grasping objects, navigating environments, and manipulating objects.

3. Autonomous driving: Reinforcement learning can be used to train autonomous vehicles to navigate complex environments and make decisions in real-time.

4. Advertising: Reinforcement learning can be used to optimize online advertising campaigns by learning which ads and targeting strategies are most effective.

5. Finance: Reinforcement learning can be used to develop trading algorithms that can learn to make profitable trades in financial markets.

6. Healthcare: Reinforcement learning can be used to develop personalized treatment plans for patients based on their medical history and other factors.

7. Resource management: Reinforcement learning can be used to optimize resource allocation in industries such as energy, transportation, and logistics.

8. Education: Reinforcement learning can be used to develop personalized learning plans for students based on their learning history and progress.

In [8]:
#Now let's ask a follow up question
chain.run(FOLLOW_UP_QUESTION)

'As an AI language model, I cannot rephrase what I just said as I do not have the capability to remember my previous statement. However, you can provide me with the statement you want me to rephrase, and I will do my best to provide an alternative wording.'

As you can see, it doesn't remember what it just responded. This proof that the LLM does NOT have memory and that we need to give the memory as a a conversation history as part of the prompt, like this:

In [9]:
hist_prompt = PromptTemplate(
    input_variables=["history", "question"],
    template="""
                {history}
                Human: {question}
                AI:
            """
    )
chain = LLMChain(llm=llm, prompt=hist_prompt)

In [10]:
Conversation_history = """
Human: {question}
AI: {response}
""".format(question=QUESTION, response=response)

In [11]:
chain.run({"history":Conversation_history, "question": FOLLOW_UP_QUESTION})

'Sure, here are some examples of how reinforcement learning can be applied:\n\n1. Game playing: Reinforcement learning algorithms can be used to create game-playing agents that can learn to play games like chess, Go, and poker.\n\n2. Robotics: Reinforcement learning can be used to train robots to perform tasks such as grasping objects, navigating environments, and manipulating objects.\n\n3. Autonomous driving: Reinforcement learning can help autonomous vehicles navigate complex environments and make real-time decisions.\n\n4. Advertising: Reinforcement learning can optimize online advertising campaigns by learning which ads and targeting strategies are most effective.\n\n5. Finance: Reinforcement learning can be used to develop trading algorithms that can learn to make profitable trades in financial markets.\n\n6. Healthcare: Reinforcement learning can be used to create personalized treatment plans for patients based on their medical history and other factors.\n\n7. Resource managemen

**Bingo!**, so we now know how to create a chatbot using LLMs, we just need to keep the state/history of the conversation and pass it as context every time

## Now that we understand the concept of memory via adding history as a context, let's go back to our GPT Smart Search engine

In order to not duplicate code, we have put many of the code used in Notebook 3 into functions. These functions are in the app/utils.py and app/prompts.py files This way we can use these functios in the app that we will build later.

In [12]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index1_name, index2_name]

agg_search_results = get_search_results(QUESTION, indexes)
ordered_results = order_search_results(agg_search_results, reranker_threshold=1)

In [13]:
docs = []
for key,value in ordered_results.items():
    for page in value["chunks"]:
        docs.append(Document(page_content=page, metadata={"source": value["location"]}))

# Calculate number of tokens of our docs
tokens_limit = model_tokens_limit(MODEL)

if(len(docs)>0):
    num_tokens = num_tokens_from_docs(docs)
    print("Custom token limit for", MODEL, ":", tokens_limit)
    print("Combined docs tokens count:",num_tokens)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")


Custom token limit for gpt-35-turbo : 3000
Combined docs tokens count: 58744


In [14]:
%%time
if num_tokens > tokens_limit:
    index = embed_docs(docs)
    top_docs = search_docs(index,QUESTION)
    
    # Now we need to recalculate the tokens count of the top results from similarity vector search
    # in order to select the chain type: stuff or map_reduce
    
    num_tokens = num_tokens_from_docs(top_docs)   
    print("Token count after similarity search:", num_tokens)
    chain_type = "map_reduce" if num_tokens > tokens_limit else "stuff"
    
else:
    # if total tokens is less than our limit, we don't need to vectorize and do similarity search
    top_docs = docs
    chain_type = "stuff"
    
print("Chain Type selected:", chain_type)

Number of chunks: 52
Token count after similarity search: 3418
Chain Type selected: map_reduce
CPU times: user 263 ms, sys: 5.28 ms, total: 268 ms
Wall time: 2.85 s


In [15]:
# Get the answer
response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type)
response['output_text']

'Reinforcement learning is finding use in many important applications, including routing, optimizing policies for tasks such as robotics, game playing, and decision making in complex environments. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0204/0204040v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

And if we ask the follow up question:

In [16]:
response = get_answer(docs=top_docs,  query=FOLLOW_UP_QUESTION, language="English",deployment=MODEL, chain_type=chain_type)
response['output_text']

'The first two excerpts do not contain relevant information to answer the question. The third excerpt discusses the differences between supervised learning and reinforcement learning. The fourth excerpt discusses the issue of reusing experience to learn about another policy in reinforcement learning. The authors present a method to estimate the value of policies using importance sampling and building a non-parametric model of the optimization surface. The paper is not concerned with optimization itself. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0204/0204040v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

Until now we just have the same as the prior Notebook 03: results from Azure Search enhanced by OpenAI model, with no memory

**Now let's add memory to it:**

Reference: https://python.langchain.com/en/latest/modules/memory/examples/adding_memory_chain_multiple_inputs.html

In [17]:
# memory object, which is neccessary to track the inputs/outputs and hold a conversation.
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question")

response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                        memory=memory)
response['output_text']

'Reinforcement learning has been successfully used in a variety of applications, including routing and playing games such as backgammon, chess, checkers, and Othello at a high level. The focus of research in reinforcement learning is on designing algorithms for an agent interacting with an environment to optimize a long-term return. While there is no specific text in the provided document that directly lists use cases for reinforcement learning, it is a versatile technique that can be applied to a wide range of problems. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0204/0204040v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

In [18]:
# Now we add a follow up question:
response = get_answer(docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                      memory=memory)
response['output_text']

'I apologize for any confusion. What I meant to say is that while there is no specific text in the provided documents that directly list use cases for reinforcement learning, it is a versatile technique that can be applied to a wide range of problems. Is there anything else I can help you with?\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0204/0204040v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf'

In [19]:
# Another follow up query
response = get_answer(docs=top_docs, query="Thank you", language="English", deployment=MODEL, chain_type=chain_type,  
                      memory=memory)
response['output_text']

"You're welcome! Is there anything else I can assist you with?\nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0204/0204040v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf"

Let's check our memory to see that it's keeping the conversation

In [20]:
memory.buffer

"Human: Tell me some use cases for reinforcement learning?\nAI: Reinforcement learning has been successfully used in a variety of applications, including routing and playing games such as backgammon, chess, checkers, and Othello at a high level. The focus of research in reinforcement learning is on designing algorithms for an agent interacting with an environment to optimize a long-term return. While there is no specific text in the provided document that directly lists use cases for reinforcement learning, it is a versatile technique that can be applied to a wide range of problems. \nSOURCES: https://demodatasetsp.blob.core.windows.net/arxivcs/0207/0207073v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0204/0204040v1.pdf, https://demodatasetsp.blob.core.windows.net/arxivcs/0105/0105027v1.pdf\nHuman: Can you rephrase what you just said?\nAI: I apologize for any confusion. What I meant to say is that while there is no specific text in the provided documents that directly lis

## Using CosmosDB as persistent memory

In previous cell we have added local RAM memory to our chatbot. However, it is not persistent, it gets deleted once the app user's session is terminated. It is necessary then to use a Database for persistent storage of each of the bot user conversations, not only for Analytics and Auditing, but also if we wisg to provide recommendations. 

Here we will store the conversation history into CosmosDB for future auditing purpose.
We will use a class in LangChain use CosmosDBChatMessageHistory, see [HERE](https://python.langchain.com/en/latest/_modules/langchain/memory/chat_message_histories/cosmos_db.html)

In [21]:
# Create CosmosDB instance from langchain cosmos class.

SESSION_ID = '001'  # this is string value, normally you need to provide the user session id.
USER_ID = 'dummy01' # this is string value, normally you need to provide the user id.

cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=AZURE_COSMOSDB_ENDPOINT,
    cosmos_database=AZURE_COSMOSDB_NAME,
    cosmos_container=AZURE_COSMOSDB_CONTAINER_NAME,
    connection_string=AZURE_COMOSDB_CONNECTION_STRING,
    session_id=SESSION_ID,
    user_id=USER_ID
    )

# prepare the cosmosdb instance
cosmos.prepare_cosmos()

In [22]:
# Create or Memory Object
memory = ConversationBufferMemory(memory_key="chat_history",input_key="question",chat_memory=cosmos)

In [23]:
# Testing using our Question
response = get_answer(docs=top_docs, query=QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                        memory=memory)
response['output_text']

'Reinforcement learning can be used in many applications, including routing. Some examples of reinforcement learning algorithms for routing are the ant-based algorithms of Subramanian et al. and the stigmergetic routing framework described in [10]. Reinforcement learning focuses on designing algorithms for an agent interacting with an environment, to adjust its behavior in such a way as to optimize a long-term return. This means searching for an optimal behavior in a class of behaviors. The interaction between agent and environment is modeled by a Markov decision process (mdp). The learning system does not know the correct behavior, or the true model of the environment it interacts with. Given the sensation of the environment state as an input, the agent chooses the action according to some rule, often called a policy. This action constitutes the output. The effectiveness of the action taken and its effect on the environment is communicated to the agent through a scalar value (reinforc

In [25]:
# Now we add a follow up question:
response = get_answer(docs=top_docs, query=FOLLOW_UP_QUESTION, language="English", deployment=MODEL, chain_type=chain_type, 
                      memory=memory)
response['output_text']

'I apologize for the confusion. In my previous response, I explained that reinforcement learning is a type of machine learning that focuses on designing algorithms for an agent interacting with an environment, to adjust its behavior in such a way as to optimize a long-term return. The interaction between agent and environment is modeled by a Markov decision process (mdp). The learning system does not know the correct behavior, or the true model of the environment it interacts with. Given the sensation of the environment state as an input, the agent chooses the action according to some rule, often called a policy. This action constitutes the output. The effectiveness of the action taken and its effect on the environment is communicated to the agent through a scalar value (reinforcement signal). As for use cases, reinforcement learning can be used in many applications, including routing. Some examples of reinforcement learning algorithms for routing are the ant-based algorithms of Subram

In [26]:
# Another follow up query
response = get_answer(docs=top_docs, query="Thank you", language="English", deployment=MODEL, chain_type=chain_type,  
                      memory=memory)
response['output_text']

"You're welcome! If you have any further questions, please let me know."

Let's check our Azure CosmosDB to see the whole conversation


In [27]:
#load message from cosmosdb
cosmos.load_messages()
cosmos.messages

[HumanMessage(content='Tell me some use cases for reinforcement learning?', additional_kwargs={}, example=False),
 AIMessage(content='Reinforcement learning can be used in many applications, including routing. Some examples of reinforcement learning algorithms for routing are the ant-based algorithms of Subramanian et al. and the stigmergetic routing framework described in [10]. Reinforcement learning focuses on designing algorithms for an agent interacting with an environment, to adjust its behavior in such a way as to optimize a long-term return. This means searching for an optimal behavior in a class of behaviors. The interaction between agent and environment is modeled by a Markov decision process (mdp). The learning system does not know the correct behavior, or the true model of the environment it interacts with. Given the sensation of the environment state as an input, the agent chooses the action according to some rule, often called a policy. This action constitutes the output. 

![CosmosDB Memory](./images/cosmos-chathistory.png)

# Summary
##### Adding memory to our application allows the user to have a conversation, however this feature is not something that comes with the LLM, but instead, memory is something that we must provide to the LLM in form of context of the question.

We added persitent memory using CosmosDB.

We also can notice that the current chain that we are using is smart, but not that much. Although we have given memory to it, it searches for similar docs everytime, it struggles to respond to prompts like: Hello, Thank you, Bye, What's your name, What's the weather and any other task that is not search in the knowledge base.



# NEXT
We know now how to do a Smart Search Engine that can power a chatbot!! great!

But, does this solve all the possible scenarios that a virtual assistant will require?  **What about if the answer to the Smart Search Engine is not related to text, but instead requires to look into tabular data?** The next notebook 05 explains and solves the tabular problem and the concept of Agents